In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('younicam-AI').getOrCreate()

spark

In [2]:
presencesDF = spark.read.json("./data/presences.json", multiLine=True)

presencesDF.show(5, False)

+------------------------+----+------------------------+------------------------+------------------------+----+-----+----+
|_id                     |aula|date                    |inDate                  |outDate                 |polo|posto|sede|
+------------------------+----+------------------------+------------------------+------------------------+----+-----+----+
|5fa8ef7d1bd2a03f4641a15e|1   |2020-11-09T07:27:57.078Z|2020-11-09T07:27:57.078Z|2020-11-09T12:05:00.362Z|1   |1    |1   |
|5fa8efa51bd2a03f4641a15f|1   |2020-11-09T07:28:37.074Z|2020-11-09T07:28:37.074Z|2020-11-09T12:05:00.363Z|1   |2    |1   |
|5fa8f0751bd2a03f4641a160|1   |2020-11-09T07:32:05.879Z|2020-11-09T07:32:05.878Z|2020-11-09T12:05:00.364Z|1   |3    |1   |
|5fa8f0811bd2a03f4641a161|1   |2020-11-09T07:32:17.390Z|2020-11-09T07:32:17.390Z|2020-11-09T07:32:20.897Z|1   |4    |1   |
|5fa8f0891bd2a03f4641a162|1   |2020-11-09T07:32:25.980Z|2020-11-09T07:32:25.980Z|2020-11-09T07:32:36.245Z|1   |5    |1   |
+---------------

In [3]:
presencesDF.count()

9841

In [4]:
presencesDF.columns

['_id', 'aula', 'date', 'inDate', 'outDate', 'polo', 'posto', 'sede']

In [5]:
presencesDF.dtypes

[('_id', 'string'),
 ('aula', 'string'),
 ('date', 'string'),
 ('inDate', 'string'),
 ('outDate', 'string'),
 ('polo', 'string'),
 ('posto', 'string'),
 ('sede', 'string')]

In [6]:
# checking for null values

from pyspark.sql.functions import isnull, when, count, col

presencesDF.select([count(when(isnull(c), c)).alias(c) for c in presencesDF.columns]).show()

+---+----+----+------+-------+----+-----+----+
|_id|aula|date|inDate|outDate|polo|posto|sede|
+---+----+----+------+-------+----+-----+----+
|  0|   0|   0|     0|    223|   0|    0|   0|
+---+----+----+------+-------+----+-----+----+



In [7]:
# delete the null values founded

presencesDF = presencesDF.replace('?', None).dropna(how='any')

presencesDF.count()

9618

In [8]:
# drop unnecessary column

presencesDF = presencesDF.drop("date")

presencesDF.columns

['_id', 'aula', 'inDate', 'outDate', 'polo', 'posto', 'sede']

In [9]:
# cast column inDate and outDate to timestamp

presencesDF = presencesDF.withColumn("inDate", presencesDF["inDate"].cast("timestamp"))

presencesDF = presencesDF.withColumn("outDate", presencesDF["outDate"].cast("timestamp"))

presencesDF


DataFrame[_id: string, aula: string, inDate: timestamp, outDate: timestamp, polo: string, posto: string, sede: string]